In [1]:
import os, glob
from pathlib import Path
import pandas as pd
import glob
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import joblib
from sklearn.metrics import (
    roc_auc_score, average_precision_score, classification_report,
    confusion_matrix, precision_recall_curve
)
import xgboost as xgb
import re

In [2]:

CANDIDATES = [
    Path("gold/model_dataset"),
    Path("datamart/gold/model_dataset"),
]

paths = []
for base in CANDIDATES:
    pat = str(base / "gold_model_dataset_*.parquet")
    found = sorted(glob.glob(pat))
    if found:
        print(f"Found {len(found)} snapshots under: {base}")
        paths = found
        break

if not paths:
    raise FileNotFoundError("Couldn’t find any gold_model_dataset_*.parquet under "
                            "'gold/model_dataset' or 'datamart/gold/model_dataset'. "
                            "Check your CWD from the diagnostics above.")

# Read each snapshot (file or parquet directory)
dfs = []
for p in paths:
    dfs.append(pd.read_parquet(p))
df = pd.concat(dfs, ignore_index=True)

# Coerce types
df["snapshot_date"] = pd.to_datetime(df["snapshot_date"])
df["label"] = df["label"].astype(int)

print("Rows:", len(df), "Cols:", df.shape[1])
display(df.head(3))


Found 24 snapshots under: datamart/gold/model_dataset
Rows: 8974 Cols: 59


,customer_id,snapshot_date,loan_id,label,label_def,interest_rate,num_credit_inquiries,credit_mix,num_of_loan,credit_history_age,...,occ_ENTREPRENEUR,occ_JOURNALIST,occ_LAWYER,occ_MANAGER,occ_MEDIA_MANAGER,occ_MECHANIC,occ_MUSICIAN,occ_SCIENTIST,occ_TEACHER,occ_WRITER
0,CUS_0X16F4,2023-07-01,CUS_0x16f4_2023_01_01,0,30dpd_6mob,6.0,1,Standard,3,362,...,0,0,0,0,1,0,0,0,0,0
1,CUS_0X4340,2023-07-01,CUS_0x4340_2023_01_01,0,30dpd_6mob,22.0,17,Bad,3,153,...,0,0,0,0,0,0,0,0,1,0
2,CUS_0X7C66,2023-07-01,CUS_0x7c66_2023_01_01,0,30dpd_6mob,17.0,6,Bad,8,242,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# --- Time-based train/valid/test/oot split -----------------------------------
from datetime import datetime

# Define the temporal boundaries
train_start = pd.Timestamp("2023-07-01")
train_end   = pd.Timestamp("2024-04-30")
valid_start = pd.Timestamp("2024-05-01")
valid_end   = pd.Timestamp("2024-06-30")
test_start  = pd.Timestamp("2024-07-01")
test_end    = pd.Timestamp("2024-10-31")
oot_start   = pd.Timestamp("2024-11-01")
oot_end     = pd.Timestamp("2024-12-31")

# Assign subsets based on snapshot_date
train_df = df[(df["snapshot_date"] >= train_start) & (df["snapshot_date"] <= train_end)]
valid_df = df[(df["snapshot_date"] >= valid_start) & (df["snapshot_date"] <= valid_end)]
test_df  = df[(df["snapshot_date"] >= test_start)  & (df["snapshot_date"] <= test_end)]
oot_df   = df[(df["snapshot_date"] >= oot_start)   & (df["snapshot_date"] <= oot_end)]

print(f"Train: {len(train_df)}, Valid: {len(valid_df)}, Test: {len(test_df)}, OOT: {len(oot_df)}")

# Drop ID-like columns
drop_cols = ["customer_id", "loan_id", "label_def"]
train_df = train_df.drop(columns=drop_cols, errors="ignore")
valid_df = valid_df.drop(columns=drop_cols, errors="ignore")
test_df  = test_df.drop(columns=drop_cols, errors="ignore")
oot_df   = oot_df.drop(columns=drop_cols, errors="ignore")

# Separate features and labels
feature_cols = [c for c in df.columns if c not in ["label","snapshot_date","customer_id","loan_id","label_def"]]

X_train, y_train = train_df[feature_cols], train_df["label"].astype(int)
X_valid, y_valid = valid_df[feature_cols], valid_df["label"].astype(int)
X_test,  y_test  = test_df[feature_cols],  test_df["label"].astype(int)
X_oot,   y_oot   = oot_df[feature_cols],   oot_df["label"].astype(int)

print(f"Feature columns: {len(feature_cols)} | Train shape: {X_train.shape}")


Train: 4978, Valid: 980, Test: 2027, OOT: 989
Feature columns: 54 | Train shape: (4978, 54)


In [4]:
# === Train Logistic Regression (Train+Valid) ================================
# Rebuild preprocessing from the detected columns
from pathlib import Path
from datetime import datetime
import re, joblib, numpy as np, pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

ARTIFACT_DIR = Path("models"); ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

def make_ohe():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

# Columns from training frame
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object","category","bool"]).columns.tolist()

# Preprocess
numeric_tf = Pipeline([("imputer", SimpleImputer(strategy="median"))])
categorical_tf = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                           ("ohe", make_ohe())])

preprocess_lr = ColumnTransformer([("num", numeric_tf, num_cols),
                                   ("cat", categorical_tf, cat_cols)])

pipe_lr = Pipeline([("prep", preprocess_lr),
                    ("clf", LogisticRegression(max_iter=1000, solver="lbfgs"))])

# Fit on Train+Valid
X_trv = pd.concat([X_train, X_valid], axis=0)
y_trv = pd.concat([y_train, y_valid], axis=0)
pipe_lr.fit(X_trv, y_trv)

# Versioned save
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
lr_path = ARTIFACT_DIR / f"logreg_pipeline_{ts}.joblib"
joblib.dump(pipe_lr, lr_path)
print("✅ Saved:", lr_path)


✅ Saved: models/logreg_pipeline_20251108_165734.joblib


/usr/local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
# === Evaluate Logistic Regression across splits===================
# Load the saved pipeline
ARTIFACT_DIR = Path("models")

def load_latest_logreg():
    # Prefer timestamped artifacts like: logreg_pipeline_YYYYMMDD_HHMMSS.joblib
    files = sorted(ARTIFACT_DIR.glob("logreg_pipeline_*.joblib"))
    if files:
        latest = files[-1]
        print("Loaded latest LR (versioned):", latest.name)
        return joblib.load(latest)
    # Fallback to non-versioned
    fallback = ARTIFACT_DIR / "logreg_pipeline.joblib"
    if fallback.exists():
        print("Loaded fallback LR:", fallback.name)
        return joblib.load(fallback)
    raise FileNotFoundError("No Logistic Regression artifacts found in models/.")

pipe_lr = load_latest_logreg()

def eval_split(model, X, y, split_name, thr=0.5):
    """Return metrics dict and print a text confusion matrix."""
    proba = model.predict_proba(X)[:, 1]
    pred  = (proba >= thr).astype(int)

    roc = roc_auc_score(y, proba)
    pr  = average_precision_score(y, proba)
    rpt = classification_report(y, pred, digits=3, output_dict=True)
    cm  = confusion_matrix(y, pred)
    tn, fp, fn, tp = cm.ravel()

    print(f"\n=== Logistic Regression — {split_name} (thr={thr:.3f}) ===")
    print(f"ROC-AUC: {roc:.3f} | PR-AUC: {pr:.3f}")
    print(f"Precision (1): {rpt['1']['precision']:.3f} | Recall (1): {rpt['1']['recall']:.3f} | F1 (1): {rpt['1']['f1-score']:.3f}")
    print("Confusion matrix:\n"
          f"TN={tn:>4}  FP={fp:>4}\n"
          f"FN={fn:>4}  TP={tp:>4}")

    return {
        "split": split_name,
        "threshold": thr,
        "roc_auc": roc,
        "pr_auc": pr,
        "precision_1": rpt["1"]["precision"],
        "recall_1": rpt["1"]["recall"],
        "f1_1": rpt["1"]["f1-score"],
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }

# 1) Evaluate at default threshold = 0.5
metrics = []
metrics.append(eval_split(pipe_lr, X_train, y_train, "Train", thr=0.5))
metrics.append(eval_split(pipe_lr, X_valid, y_valid, "Valid", thr=0.5))
metrics.append(eval_split(pipe_lr, X_test,  y_test,  "Test",  thr=0.5))
metrics.append(eval_split(pipe_lr, X_oot,   y_oot,   "OOT",   thr=0.5))

# 2) Tune threshold on VALID (maximize F1), then re-evaluate all splits
proba_valid = pipe_lr.predict_proba(X_valid)[:, 1]
prec_v, rec_v, thr_v = precision_recall_curve(y_valid, proba_valid)
f1_v = 2 * (prec_v * rec_v) / (prec_v + rec_v + 1e-9)
best_idx = np.nanargmax(f1_v)
best_thr = float(thr_v[best_idx])
print(f"\n>>> Best threshold from VALID (max F1): {best_thr:.3f} "
      f"| Precision={prec_v[best_idx]:.3f}, Recall={rec_v[best_idx]:.3f}, F1={f1_v[best_idx]:.3f}")

metrics.append(eval_split(pipe_lr, X_train, y_train, "Train_tuned", thr=best_thr))
metrics.append(eval_split(pipe_lr, X_valid, y_valid, "Valid_tuned", thr=best_thr))
metrics.append(eval_split(pipe_lr, X_test,  y_test,  "Test_tuned",  thr=best_thr))
metrics.append(eval_split(pipe_lr, X_oot,   y_oot,   "OOT_tuned",   thr=best_thr))

# 3) Save metrics (optional)
metrics_df = pd.DataFrame(metrics)
display(metrics_df)
metrics_df.to_csv("models/logreg_metrics_all_splits.csv", index=False)
print("\nSaved metrics to models/logreg_metrics_all_splits.csv")


Loaded latest LR (versioned): logreg_pipeline_20251108_165734.joblib

=== Logistic Regression — Train (thr=0.500) ===
ROC-AUC: 0.717 | PR-AUC: 0.453
Precision (1): 0.488 | Recall (1): 0.196 | F1 (1): 0.280
Confusion matrix:
TN=3300  FP= 286
FN=1119  TP= 273

=== Logistic Regression — Valid (thr=0.500) ===
ROC-AUC: 0.684 | PR-AUC: 0.423
Precision (1): 0.381 | Recall (1): 0.136 | F1 (1): 0.201
Confusion matrix:
TN= 621  FP=  65
FN= 254  TP=  40

=== Logistic Regression — Test (thr=0.500) ===
ROC-AUC: 0.704 | PR-AUC: 0.441
Precision (1): 0.453 | Recall (1): 0.170 | F1 (1): 0.247
Confusion matrix:
TN=1303  FP= 123
FN= 499  TP= 102

=== Logistic Regression — OOT (thr=0.500) ===
ROC-AUC: 0.743 | PR-AUC: 0.530
Precision (1): 0.518 | Recall (1): 0.188 | F1 (1): 0.275
Confusion matrix:
TN= 632  FP=  53
FN= 247  TP=  57

>>> Best threshold from VALID (max F1): 0.241 | Precision=0.477, Recall=0.704, F1=0.569

=== Logistic Regression — Train_tuned (thr=0.241) ===
ROC-AUC: 0.717 | PR-AUC: 0.453
Pre

,split,threshold,roc_auc,pr_auc,precision_1,recall_1,f1_1,tn,fp,fn,tp
0,Train,0.500000,0.716992,0.452506,0.488372,0.196121,0.279856,3300,286,1119,273
1,Valid,0.500000,0.684462,0.422762,0.380952,0.136054,0.200501,621,65,254,40
2,Test,0.500000,0.703522,0.440685,0.453333,0.169717,0.246973,1303,123,499,102
3,OOT,0.500000,0.742802,0.529626,0.518182,0.187500,0.275362,632,53,247,57
4,Train_tuned,0.240939,0.716992,0.452506,0.444924,0.739943,0.555705,2301,1285,362,1030
5,Valid_tuned,0.240939,0.684462,0.422762,0.476959,0.704082,0.568681,459,227,87,207
6,Test_tuned,0.240939,0.703522,0.440685,0.459893,0.715474,0.559896,921,505,171,430
7,OOT_tuned,0.240939,0.742802,0.529626,0.495726,0.763158,0.601036,449,236,72,232



Saved metrics to models/logreg_metrics_all_splits.csv


In [6]:
# === Train XGBoost (DMatrix API with early stopping), then refit on Train+Valid ===
ARTIFACT_DIR = Path("models"); ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

def make_ohe():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

# Columns
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object","category","bool"]).columns.tolist()

# Preprocess pieces
numeric_tf = Pipeline([("imputer", SimpleImputer(strategy="median"))])
categorical_tf = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                           ("ohe", make_ohe())])

# ---- Stage 1: early stopping on VALID
pre_stage1 = ColumnTransformer([("num", numeric_tf, num_cols),
                                ("cat", categorical_tf, cat_cols)])
Xtr = pre_stage1.fit_transform(X_train, y_train)
Xva = pre_stage1.transform(X_valid)

dtr = xgb.DMatrix(Xtr, label=y_train.to_numpy())
dva = xgb.DMatrix(Xva, label=y_valid.to_numpy())

params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "eta": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "tree_method": "hist",
    "seed": 42,
}
bst_es = xgb.train(params, dtr, num_boost_round=2000, evals=[(dva, "valid")],
                   early_stopping_rounds=50, verbose_eval=False)
best_iter = int(bst_es.best_iteration + 1)
print("Best iteration:", best_iter)

# ---- Stage 2: refit on Train+Valid
pre_final = ColumnTransformer([("num", numeric_tf, num_cols),
                               ("cat", categorical_tf, cat_cols)])
X_trv = pd.concat([X_train, X_valid], axis=0)
y_trv = pd.concat([y_train, y_valid], axis=0)

Xtrv = pre_final.fit_transform(X_trv, y_trv)
dtrv = xgb.DMatrix(Xtrv, label=y_trv.to_numpy())

bst_final = xgb.train(params, dtrv, num_boost_round=best_iter, verbose_eval=False)

# Versioned save
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
xgb_path = ARTIFACT_DIR / f"xgb_bundle_{ts}.joblib"
bundle = {"preprocess": pre_final, "booster": bst_final,
          "num_cols": num_cols, "cat_cols": cat_cols, "best_iter": best_iter}
joblib.dump(bundle, xgb_path)
print("✅ Saved:", xgb_path)


Best iteration: 70
✅ Saved: models/xgb_bundle_20251108_170307.joblib


In [7]:
# === Evaluate XGBoost (Booster + preprocess) across splits, no plots ===========
ARTIFACT_DIR = Path("models")

def load_latest_xgb():
    """Load the most recent XGBoost bundle from models/."""
    files = sorted(ARTIFACT_DIR.glob("xgb_bundle_*.joblib"))
    if files:
        latest = files[-1]
        print("Loaded latest XGB (versioned):", latest.name)
        return joblib.load(latest)
    fallback = ARTIFACT_DIR / "xgb_bundle.joblib"
    if fallback.exists():
        print("Loaded fallback XGB:", fallback.name)
        return joblib.load(fallback)
    raise FileNotFoundError("No XGBoost artifacts found in models/.")

# Load latest model bundle
bundle = load_latest_xgb()
prep = bundle["preprocess"]
booster = bundle["booster"]

def predict_proba(prep, booster, X):
    Xt = prep.transform(X)
    d = xgb.DMatrix(Xt)
    return booster.predict(d)  # probability for class 1

def eval_split(prep, booster, X, y, split_name, thr=0.5):
    proba = predict_proba(prep, booster, X)
    pred  = (proba >= thr).astype(int)

    roc = roc_auc_score(y, proba)
    pr  = average_precision_score(y, proba)
    rpt = classification_report(y, pred, digits=3, output_dict=True)
    cm  = confusion_matrix(y, pred)
    tn, fp, fn, tp = cm.ravel()

    print(f"\n=== XGBoost — {split_name} (thr={thr:.3f}) ===")
    print(f"ROC-AUC: {roc:.3f} | PR-AUC: {pr:.3f}")
    print(f"Precision (1): {rpt['1']['precision']:.3f} | Recall (1): {rpt['1']['recall']:.3f} | F1 (1): {rpt['1']['f1-score']:.3f}")
    print("Confusion matrix:\n"
          f"TN={tn:>4}  FP={fp:>4}\n"
          f"FN={fn:>4}  TP={tp:>4}")

    return {
        "split": split_name, "threshold": thr,
        "roc_auc": roc, "pr_auc": pr,
        "precision_1": rpt["1"]["precision"],
        "recall_1": rpt["1"]["recall"],
        "f1_1": rpt["1"]["f1-score"],
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)
    }

# 1) Evaluate at threshold = 0.5
metrics_xgb = []
metrics_xgb.append(eval_split(prep, booster, X_train, y_train, "Train", thr=0.5))
metrics_xgb.append(eval_split(prep, booster, X_valid, y_valid, "Valid", thr=0.5))
metrics_xgb.append(eval_split(prep, booster, X_test,  y_test,  "Test",  thr=0.5))
metrics_xgb.append(eval_split(prep, booster, X_oot,   y_oot,   "OOT",   thr=0.5))

# 2) Tune threshold on VALID (maximize F1), then re-evaluate all splits
proba_valid = predict_proba(prep, booster, X_valid)
prec_v, rec_v, thr_v = precision_recall_curve(y_valid, proba_valid)
f1_v = 2 * (prec_v * rec_v) / (prec_v + rec_v + 1e-9)
best_idx = np.nanargmax(f1_v)
best_thr = float(thr_v[best_idx])
print(f"\n>>> Best threshold from VALID (max F1): {best_thr:.3f} "
      f"| Precision={prec_v[best_idx]:.3f}, Recall={rec_v[best_idx]:.3f}, F1={f1_v[best_idx]:.3f}")

metrics_xgb.append(eval_split(prep, booster, X_train, y_train, "Train_tuned", thr=best_thr))
metrics_xgb.append(eval_split(prep, booster, X_valid, y_valid, "Valid_tuned", thr=best_thr))
metrics_xgb.append(eval_split(prep, booster, X_test,  y_test,  "Test_tuned",  thr=best_thr))
metrics_xgb.append(eval_split(prep, booster, X_oot,   y_oot,   "OOT_tuned",   thr=best_thr))

# 3) Save metrics
metrics_xgb_df = pd.DataFrame(metrics_xgb)
display(metrics_xgb_df)
metrics_xgb_df.to_csv("models/xgb_metrics_all_splits.csv", index=False)
print("\nSaved metrics to models/xgb_metrics_all_splits.csv")


Loaded latest XGB (versioned): xgb_bundle_20251108_170307.joblib

=== XGBoost — Train (thr=0.500) ===
ROC-AUC: 0.902 | PR-AUC: 0.801
Precision (1): 0.768 | Recall (1): 0.622 | F1 (1): 0.688
Confusion matrix:
TN=3325  FP= 261
FN= 526  TP= 866

=== XGBoost — Valid (thr=0.500) ===
ROC-AUC: 0.896 | PR-AUC: 0.810
Precision (1): 0.789 | Recall (1): 0.599 | F1 (1): 0.681
Confusion matrix:
TN= 639  FP=  47
FN= 118  TP= 176

=== XGBoost — Test (thr=0.500) ===
ROC-AUC: 0.757 | PR-AUC: 0.555
Precision (1): 0.600 | Recall (1): 0.507 | F1 (1): 0.550
Confusion matrix:
TN=1223  FP= 203
FN= 296  TP= 305

=== XGBoost — OOT (thr=0.500) ===
ROC-AUC: 0.781 | PR-AUC: 0.615
Precision (1): 0.634 | Recall (1): 0.497 | F1 (1): 0.557
Confusion matrix:
TN= 598  FP=  87
FN= 153  TP= 151

>>> Best threshold from VALID (max F1): 0.199 | Precision=0.610, Recall=0.878, F1=0.720

=== XGBoost — Train_tuned (thr=0.199) ===
ROC-AUC: 0.902 | PR-AUC: 0.801
Precision (1): 0.579 | Recall (1): 0.881 | F1 (1): 0.699
Confusion 

,split,threshold,roc_auc,pr_auc,precision_1,recall_1,f1_1,tn,fp,fn,tp
0,Train,0.500000,0.901888,0.800592,0.768412,0.622126,0.687574,3325,261,526,866
1,Valid,0.500000,0.895911,0.810426,0.789238,0.598639,0.680851,639,47,118,176
2,Test,0.500000,0.757078,0.554729,0.600394,0.507488,0.550045,1223,203,296,305
3,OOT,0.500000,0.781060,0.614728,0.634454,0.496711,0.557196,598,87,153,151
4,Train_tuned,0.198991,0.901888,0.800592,0.578501,0.881466,0.698548,2692,894,165,1227
5,Valid_tuned,0.198991,0.895911,0.810426,0.609929,0.877551,0.719665,521,165,36,258
6,Test_tuned,0.198991,0.757078,0.554729,0.493228,0.727121,0.587761,977,449,164,437
7,OOT_tuned,0.198991,0.781060,0.614728,0.519016,0.763158,0.617843,470,215,72,232



Saved metrics to models/xgb_metrics_all_splits.csv
